# Track Cluster

In [1]:
import pandas as pd
import numpy as np
from utils import utils
from utils import myplot
df = pd.read_csv(r'wifi_track_data\dacang\track_data\dacang_track_data_3_final_1220.csv')
df.t = pd.to_datetime(df.t)
mac_list = df.m.unique()
df_wifipos = pd.read_csv(r'wifi_track_data\dacang\pos_data\wifi_pos_new_3_1220.csv')
from tqdm import tqdm
from importlib import reload
from grid_world.experts import Experts

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

## 特征聚类

In [2]:
#获取专家轨迹
experts = Experts(40,30,trajs_file_path = r"wifi_track_data\dacang\track_data\trajs_0117_40x30.csv",)
_,x,y = experts.GetExpertsMovingCenter()

Get experts trajs num:1343


In [3]:
mac_list = df['m'].unique()
ori_mac_list = df.oriMac.unique()
ori_Mac = []
data_count = []
Dur = []
wifi_count = []
active_moment = []
middle_moment = []
moving_dis = []
for m in mac_list:
    ori_Mac.append(m.split('-')[1])
    df_now = utils.GetDfNow(df, m)
    data_count.append(len(df_now))
    t1 = df_now.iloc[0].t
    t2 = df_now.iloc[len(df_now)-1].t
    t1 = t1.hour + t1.minute/60 + t1.second/3600
    t2 = t2.hour + t2.minute/60 + t2.second/3600
    Dur.append(t2-t1)
    middle_moment.append((t2+t1)/2)
    wifi_count.append(len(df_now['a'].unique()))
    h = 0
    row_last = df_now.iloc[0]
    dis = 0
    for i,row in df_now.iterrows():
        h += row.t.hour + row.t.minute/60 + row.t.second/3600
        if row.a != row_last.a:
            dis += utils.GetWifiTrackDistance(row.a,row_last.a,df_wifipos)
        row_last = row
    active_moment.append(h/len(df_now))
    moving_dis.append(dis)

df_sta = pd.DataFrame({'mac':mac_list,
                       'ori_mac':ori_Mac,
                       'data_count':data_count,#数据量
                       'Dur':Dur,#持续时间
                       'trajs_len':experts.traj_all_lens,#轨迹长度
                       'wifi_count':wifi_count,#经过的wifi数量
                       'active_moment':active_moment,#活跃时间
                       'moving_dis':moving_dis,#移动距离
                       'middle_moment':middle_moment,#中间时刻
                        'center_x':x,
                        'center_y':y
                       })
df_sta

,mac,ori_mac,data_count,Dur,trajs_len,wifi_count,active_moment,moving_dis,middle_moment,center_x,center_y
0,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",5,7.137500,62,3,21.925500,255.84,20.428194,27.870968,17.306452
1,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",9,7.140278,127,5,22.534414,437.00,20.429583,27.267717,17.614173
2,"7.5-20,178,229,137,47,142","20,178,229,137,47,142",12,0.133056,292,2,23.961944,697.56,23.928750,27.797945,20.910959
3,"7.6-48,74,38,155,214,98","48,74,38,155,214,98",44,23.999167,376,14,4.545069,1053.48,12.000139,28.212766,18.688830
4,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",11,23.996389,167,4,10.667727,161.62,11.998750,26.598802,17.383234
...,...,...,...,...,...,...,...,...,...,...,...
1338,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",14,16.207500,532,4,15.388036,2014.83,15.774306,16.954887,11.554511
1339,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",32,13.351667,559,6,16.939488,2149.56,17.204444,14.237925,10.359571
1340,"8.7-56,251,20,50,26,70","56,251,20,50,26,70",6,8.996944,106,3,18.492269,363.15,15.832361,22.877358,15.603774
1341,"8.7-144,173,247,200,14,93","144,173,247,200,14,93",10,8.199167,131,4,18.670944,559.52,15.882361,21.748092,14.099237


In [23]:
myplot.Boxes([(df_sta.data_count,"数据量"),
              (df_sta.active_moment,"平均活跃时刻"),
              (df_sta.moving_dis,"移动距离"),
              (df_sta.middle_moment,"中间时刻"),
              (df_sta.Dur,'持续时间')
              ])

## 清除离群值
随机孤立森林

In [24]:
#归一化
parse_list = ['data_count',
              'active_moment',
              'moving_dis',
              'middle_moment',
              'Dur'
              ]
df_outlier = df_sta[parse_list]
df_outlier = utils.Normalize_df(df_outlier,parse_list,mul_index=10)
df_outlier

,data_count,active_moment,moving_dis,middle_moment,Dur
0,0.140845,9.132320,0.281856,8.488534,2.971261
1,0.422535,9.391764,0.481438,8.489134,2.972419
2,0.633803,10.000000,0.768493,10.000000,0.051526
3,2.887324,1.726939,1.160606,4.849477,9.999884
4,0.563380,4.335655,0.178055,4.848877,9.998726
...,...,...,...,...,...
1338,0.774648,6.346865,2.219714,6.479083,6.752003
1339,2.042254,7.007901,2.368144,7.096587,5.561577
1340,0.211268,7.669503,0.400078,6.504150,3.746353
1341,0.492958,7.745633,0.616416,6.525739,3.413807


In [25]:
cols = {'data_count':'数据量','active_moment':'平均活跃时刻','moving_dis':'移动距离','middle_moment':'中间时刻','Dur':'持续时间'}
df_outlier = df_outlier.rename(columns=cols)
df_outlier

,数据量,平均活跃时刻,移动距离,中间时刻,持续时间
0,0.140845,9.132320,0.281856,8.488534,2.971261
1,0.422535,9.391764,0.481438,8.489134,2.972419
2,0.633803,10.000000,0.768493,10.000000,0.051526
3,2.887324,1.726939,1.160606,4.849477,9.999884
4,0.563380,4.335655,0.178055,4.848877,9.998726
...,...,...,...,...,...
1338,0.774648,6.346865,2.219714,6.479083,6.752003
1339,2.042254,7.007901,2.368144,7.096587,5.561577
1340,0.211268,7.669503,0.400078,6.504150,3.746353
1341,0.492958,7.745633,0.616416,6.525739,3.413807


In [27]:
parse_list = ['数据量','平均活跃时刻','移动距离','中间时刻','持续时间']

In [28]:
from sklearn.ensemble import IsolationForest
rng = np.random.RandomState(42)
model_IF = IsolationForest(max_samples=2000,random_state=rng,contamination='auto')
model_IF.fit(df_outlier)
df_outlier['label'] = model_IF.predict(df_outlier[parse_list])
print(df_outlier.label.value_counts())

myplot.Scatter_Matrix(df_outlier,
                      dimentions = parse_list,
                        color = 'label')

label
 1    1193
-1     150
Name: count, dtype: int64


In [34]:
parse_list = ['data_count',
              'active_moment',
              'moving_dis',
              'middle_moment',
              'Dur'
              ]
df_cluster = df_sta[['mac']+parse_list][df_outlier.label == 1].reset_index(drop=True)
parse_list = ['数据量','平均活跃时刻','移动距离','中间时刻','持续时间']
df_cluster = df_cluster.rename(columns=cols)
df_cluster = utils.Normalize_df(df_cluster,parse_list,mul_index=10)

df_cluster

,mac,数据量,平均活跃时刻,移动距离,中间时刻,持续时间
0,"7.5-48,74,38,155,214,98",0.666667,10.000000,1.006004,9.375206,2.944233
1,"7.6-48,74,38,155,214,98",4.555556,0.534807,2.425183,4.621536,9.999884
2,"7.6-32,50,51,210,96,61",0.888889,3.756278,0.372060,4.620753,9.998721
3,"7.6-48,74,38,133,29,213",1.333333,3.149364,1.340425,4.620753,9.998721
4,"7.6-20,178,229,137,47,142",0.333333,4.024424,0.258752,4.620205,9.997675
...,...,...,...,...,...,...
1188,"8.7-76,2,32,111,66,59",0.333333,5.327560,0.888667,4.961621,4.120485
1189,"8.7-252,4,28,35,119,153",1.222222,6.239894,4.638276,6.749926,6.738976
1190,"8.7-124,161,119,201,50,40",3.222222,7.056199,4.948434,7.556433,5.543775
1191,"8.7-56,251,20,50,26,70",0.333333,7.873203,0.835996,6.782665,3.721271


In [35]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = parse_list)

## 简单的居民、游客、路过人群分类
- 当在村庄天数超过15天被视为居民
- 当一天内待在区域超过10小时的天数超过5天时，被视为居民
- 当一天内待在区域小于0.5小时的天数超过5天时，被视为路过
- 当任意一天停留时间超过10小时被视为长住游客
- 其余被视为短期游客

In [36]:
def DivideId(df_now):
    days = df_now.calender.unique()
    count = 0
    isPassing = 0
    for i in range(len(days)):
        df_one = df_now[df_now.calender == days[i]]
        time = df_one.iloc[len(df_one)-1].t - df_one.iloc[0].t
        h = time.components.hours
        if h>10:
            count += 1
        elif h<0.5:
            isPassing += 1
    if len(days) > 15:
        return 'resident'
    if isPassing > 5:
        return 'pass'
    elif count < 5 and count>0:
        return 'tourist_long'
    elif count == 0:
        return 'tourist_short'
    else:
        return 'resident'
    

tourist_long_list = []
tourist_short_list = []
resident_list = []
pass_list = []

for mac in tqdm(ori_mac_list,desc="Dividing:"):
    df_now = df[df.oriMac == mac]
    df_now.sort_values(by='t')
    result = DivideId(df_now)
    if result == 'pass':
        pass_list.append(mac)
    elif result == 'tourist_long':
        tourist_long_list.append(mac)
    elif result == 'tourist_short':
        tourist_short_list.append(mac)
    else:
        resident_list.append(mac)

# add id to df_sta
def GetId(m):
    if m in tourist_long_list:
        return 1
    elif m in tourist_short_list:
        return 2
    elif m in resident_list:
        return 0
    else:
        return -1
df_sta['id'] = df_sta.ori_mac.apply(GetId)
df_cluster['id'] = df_sta.ori_mac.apply(GetId)

print('pass count:',len(pass_list))
print('long tourists count:',len(tourist_long_list))
print('short tourists count:',len(tourist_short_list))
print('residents count:',len(resident_list))


Dividing:: 100%|██████████| 315/315 [00:00<00:00, 351.46it/s]


pass count: 0
long tourists count: 142
short tourists count: 108
residents count: 65


In [37]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = parse_list,
                        color = 'id',
                        color_scale='Jet')

## K-means++聚类

In [29]:
#获得居民最佳聚类数
df_score_resi = utils.GetKmeansClusterNumDf(df_cluster,parse_list)
myplot.Double_Axes_Line(df_score_resi,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [30]:
df_cluster['cluster'] = utils.Kmeans(df_cluster,parse_list,3)
myplot.Scatter_Matrix(df_cluster,
                      dimentions = parse_list,
                        color = 'cluster',
                        color_scale='Jet')
df_cluster.cluster.value_counts()

cluster
2    615
0    379
1    222
Name: count, dtype: int64

In [31]:
df_cluster

,mac,trajs_len,active_moment,moving_dis,middle_moment,id,cluster
0,"7.6-48,74,38,155,214,98",2.451613,0.547978,2.515142,4.621536,0,2
1,"7.6-32,50,51,210,96,61",0.953405,3.848784,0.385861,4.620753,0,2
2,"7.6-48,74,38,133,29,213",1.870968,3.226924,1.390147,4.620753,0,2
3,"7.6-20,178,229,137,47,142",0.315412,4.123534,0.268351,4.620205,0,2
4,"7.6-4,207,140,11,128,186",1.318996,2.866861,0.526005,4.621771,0,2
...,...,...,...,...,...,...,...
1211,"8.7-8,250,121,45,167,67",2.086022,4.087606,2.069356,6.412347,2,2
1212,"8.7-76,2,32,111,66,59",0.623656,5.458763,0.921632,4.961621,0,2
1213,"8.7-252,4,28,35,119,153",3.569892,6.393564,4.810328,6.749926,1,1
1214,"8.7-56,251,20,50,26,70",0.516129,8.067097,0.867006,6.782665,0,0


In [32]:
#将聚类结果加入到df
cluster = []
for i,r in df.iterrows():
    if r.m in df_cluster.mac.values:
        cluster.append(df_cluster[df_cluster.mac == r.m].cluster.values[0])
    else:
        cluster.append(-1)
df['cluster'] = cluster

In [33]:
df.to_csv(r'wifi_track_data\dacang\track_data\dacang_track_data_5_clustered_0203.csv',index=False)

In [34]:
df_cluster_result = df_cluster[['mac','cluster']]
df_cluster_result.to_csv(r'wifi_track_data\dacang\cluster_data\cluster_result_0203.csv',index=False)

## 居民、游客单独聚类

In [59]:
#获得居民最佳聚类数
df_resident = df_cluster[df_cluster.id == 0]
df_score_resi = utils.GetKmeansClusterNumDf(df_resident,parse_list)
myplot.Double_Axes_Line(df_score_resi,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [60]:
#获得游客最佳聚类数
df_tourist = df_cluster[df_cluster.id == 1]
df_score_tour = utils.GetKmeansClusterNumDf(df_tourist,parse_list)
myplot.Double_Axes_Line(df_score_tour,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [61]:
df_resident['cluster'] = utils.Kmeans(df_resident,parse_list,3)
myplot.Scatter_Matrix(df_resident,
                      dimentions = parse_list,
                        color = 'cluster',
                        color_scale='Jet')

In [63]:
df_tourist['cluster'] = utils.Kmeans(df_tourist,parse_list,3)
myplot.Scatter_Matrix(df_tourist,
                      dimentions = parse_list,
                        color = 'cluster',
                        color_scale='Jet')

In [44]:
df_resident.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_resident.csv",index=False)
df_tourist.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_tourist.csv",index=False)